# Introduction
Building on the model visualization of Mol et al., the acetone production pathway is added 

In [1]:
import cameo
import pandas as pd
import cobra.io
import escher
from escher import Builder
from cobra import Reaction, Metabolite
from os.path import dirname, join, pardir

In [2]:
data_dir = "/Users/phivri/Documents/GitHub/MoGeoModel"
model = cobra.io.read_sbml_model(join(data_dir,'Models/p-thermo_acetone_anaerobic.xml'))

In [6]:
medium = model.medium
medium["EX_o2_e"] = 1000.0
medium["EX_ac_e"] = 10.0
medium["EX_sucr_e"] = 0.0
medium["EX_glc__D_e"] = 0.0
model.medium = medium

In [7]:
solution = model.optimize()
Builder(model = model, map_json = join(data_dir,'Models/p-thermo_acetone_map.json'), reaction_data = solution.fluxes.to_dict())

Builder(reaction_data={'IDPh': 10.436720375457611, 'CAT': 0.0, 'PDHam1hi': 0.0, 'HYDA': 0.0, 'MALHYDRO': 0.0, …

In [8]:
Y_XS_max = solution.fluxes.EX_Biomass_e/solution.fluxes.EX_ac_e

print(f"The maximum yield on acetate is {Y_XS_max}")

ratio = 0.8

print(f"The acetone producing strain is expected to grow to a minimum of {ratio*100}% of its maximum Biomass yield")

The maximum yield on acetate is -0.013191308096901149
The acetone producing strain is expected to grow to a minimum of 80.0% of its maximum Biomass yield


In [14]:
MaxGrowth = model.problem.Constraint(
    model.reactions.EX_Biomass_e.flux_expression - Y_XS_max * ratio * model.reactions.EX_ac_e.flux_expression,
    lb=0,
    ub=1000)
model.add_cons_vars(MaxGrowth)

model.objective = 'EX_act_e'

solution = model.optimize()

model.summary()

In [15]:
Builder(model = model, map_json = join(data_dir,'Models/p-thermo_acetone_map.json'), reaction_data = solution.fluxes.to_dict())

Builder(reaction_data={'IDPh': 9.557005568658845, 'CAT': 0.0, 'PDHam1hi': 0.0, 'HYDA': 0.0, 'MALHYDRO': 0.0, '…

In [32]:
solution = model.optimize()
Builder(model = model, map_json = join(data_dir,'Models/AcetonePathway.json'), reaction_data = solution.fluxes.to_dict())

Builder(reaction_data={'IDPh': 9.557005890465334, 'CAT': -8.225722834710502e-09, 'PDHam1hi': 0.0, 'HYDA': 0.0,…

In [33]:
model.metabolites.atp_c.summary()

Percent,Flux,Reaction,Definition
85.38%,34.7,ATPS4r,adp_c + 4.0 h_e + pi_c <=> atp_c + h2o_c + 3.0 h_c
0.00%,0.0006265,CYTK2,atp_c + dcmp_c <=> adp_c + dcdp_c
14.62%,5.943,SUCOAS,atp_c + coa_c + succ_c <=> adp_c + pi_c + succoa_c
0.00%,0.0008171,URIDK2m,atp_c + dump_c <=> adp_c + dudp_c
Percent,Flux,Reaction,Definition
0.38%,-0.1552,ACCOAC,accoa_c + atp_c + hco3_c --> adp_c + malcoa_c + pi_c
22.92%,-9.316,ACS,ac_c + atp_c + coa_c --> accoa_c + amp_c + ppi_c
23.35%,-9.491,ADK1,amp_c + atp_c <=> 2.0 adp_c
0.05%,-0.02024,ADSK,aps_c + atp_c --> adp_c + h_c + paps_c
0.00%,-2.101E-05,AFAT,atp_c + fmn_c --> fad_c + ppi_c


In [29]:
model.metabolites.accoa_c.summary()

Percent,Flux,Reaction,Definition
92.16%,9.316,ACS,ac_c + atp_c + coa_c --> accoa_c + amp_c + ppi_c
0.00%,7.029E-05,PAI2T,coa_c + dhptdd_c + h_c --> accoa_c + dhap_c
7.84%,0.7924,ctfAB_ac,aacoa_c + ac_c --> acac_c + accoa_c + h_c
Percent,Flux,Reaction,Definition
0.01%,-0.001444,ACALD,acald_c + coa_c + nad_c <=> accoa_c + h_c + nadh_c
1.54%,-0.1552,ACCOAC,accoa_c + atp_c + hco3_c --> adp_c + malcoa_c + pi_c
0.00%,-6.612E-06,ACCOAT,accoa_c + gam1p_c --> aagamp_c + coa_c
0.33%,-0.03347,APATi,accoa_c + h2o_c + thdp_c --> acamoxm_c + coa_c
70.67%,-7.143,CS,cit_c + coa_c + h_c <=> accoa_c + h2o_c + oaa_c
0.09%,-0.0088,HSERTA,accoa_c + hom__L_c --> achms_c + coa_c
